In [2]:
import pandas as pd

In [3]:
# Especifica la ruta del archivo Excel
archivo_excel = 'pruebas/mayoristas_junio_26_2018.xlsx'
archivo_excel1 = 'pruebas/anex-SIPSADiario-29sep2023.xlsx'

# Lee el archivo Excel en un DataFrame de Pandas
df = pd.read_excel(archivo_excel,header=2)
df1 = pd.read_excel(archivo_excel1,header=2)

# Ahora puedes trabajar con el DataFrame 'df'
# Por ejemplo, imprimir las primeras filas del DataFrame
#df.iloc[:, [0, 21]]
dflimpio1 =df.loc[:, ["Precio $/Kg","Tunja"]]
dflimpio1 =dflimpio1.rename(columns={ 'Tunja': 'junio_26_2018'})
dflimpio2 =df1.loc[:, ["Precio $/Kg","Tunja"]]

dflimpio1 = dflimpio1.drop([0,1,13,33,40,41,42])
dflimpio2 = dflimpio2.drop([0,1,13,33,40,41,42])


dfinal  =pd.merge(dflimpio1, dflimpio2, on='Precio $/Kg', how='left') 
dfinal


,Precio $/Kg,junio_26_2018,Tunja
0,Ahuyama,609,667
1,Arveja verde en vaina,3480,4067
2,Cebolla cabezona blanca,380,2275
3,Cebolla junca,883,2375
4,Chocolo mazorca,1150,NaN
5,Fríjol verde*,2893,3667
6,Habichuela,2000,4067
7,Lechuga batavia,1933,n.d.
8,Pepino cohombro,1100,1573
9,Pimentón,2262,2018


In [4]:
import requests
global extensionglobal
global nombre
# Función para descargar el archivo dado un URL
def descargar_excel(url):
    global extensionglobal
    extensions = ['xlsx', 'xls']

    
    for ext in extensions:
        try_url = url.replace('.xlsx', f'.{ext}')
        response = requests.get(try_url)
        if response.status_code == 200:
            #respuesta=[]
            extensionglobal = try_url
            return response.content
    
    print(f"No se pudo descargar el archivo del URL: {url}")
    return "1"

# Generar URLs basados en el patrón dado
def generar_urls():
    global nombre
    urls = []
    #meses para el primer formato
    meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
             'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
    #meses para el segundo formato
    meses2 = ['ene', 'feb', 'mar', 'abr', 'may', 'jun',
             'jul', 'ago', 'sep', 'oct', 'nov', 'dic']
    
    anos = [str(a) for a in range(2018, 2025)]  # años entre 2018 y 2022

    dias = [str(d).zfill(2) for d in range(1, 32)]  # días del mes (01 a 31)
    diasformato = ['01','02','03','04','05','06','07','08','09']
    # Bandera para indicar si se ha alcanzado el punto de inicio
    start_generating = False
    #primer formato
    for ano in anos:
        for mes in meses:
            #caso en el que terminan en 1 en vez de 01
            if (ano == '2018') and (mes=='enero' or mes=='febrero' or mes=='marzo' or mes=='abril'):
                #dias sin el 0 en los digitos individuales
                dias = [str(d) for d in range(1, 32)] 
            else:  
                dias = [str(d).zfill(2) for d in range(1, 32)]  # días del mes (01 a 31)
            for dia in dias:
                if ano == '2018' and mes == 'mayo' and dia == "08":
                    start_generating = True
                if start_generating:
                    url = [f"https://www.dane.gov.co/files/investigaciones/agropecuario/sipsa/mayoristas_{mes}_{dia}_{ano}.xlsx",'1']
                    urls.append(url)
                if ano == '2022' and mes == 'enero' and dia == "31":
                    start_generating = False
   
    #segundo formato
    for ano in anos:
        for mes in meses2:
            for dia in dias:
                if ano == '2022' and mes == 'feb':
                    start_generating = True
                #caso raro de may_feb en febrero 2022
                if start_generating and dia in diasformato and ano == '2022' and mes == 'feb':
                    url = [f"https://www.dane.gov.co/files/investigaciones/agropecuario/sipsa/anex_may_{mes}_{dia}_{ano}.xlsx", "2"]
                    urls.append(url)  
                #caso estandar
                elif start_generating:
                    url = [f"https://www.dane.gov.co/files/investigaciones/agropecuario/sipsa/anex_{mes}_{dia}_{ano}.xlsx", "2"]
                    urls.append(url)  
                if ano == '2023' and mes == 'may' and dia == "31":
                    start_generating = False  
    #tercer formato

    for ano in anos:
        for mes in meses2:
            for dia in dias:
                if ano == '2023' and mes == 'jun':
                    start_generating = True
                if start_generating:
                    url = [f'https://www.dane.gov.co/files/operaciones/SIPSA/anex-SIPSADiario-{dia}{mes}{ano}.xlsx','3']
                    urls.append(url)   
    return urls

#diccionarios para remplazar los respectivos datos
remplazar = {
    'ene': '01', 
    'feb': '02', 
    'mar': '03',
    'abr': '04',
    'may': '05',
    'jun': '06',
    'jul': '07',
    'ago': '08',
    'sep': '09',
    'oct': '10',
    'nov': '11',
    'dic': '12'}
remplazar2 = { 
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}
# Probar descargando algunos archivos de ejemplo
urls = generar_urls()[:30000]  # Descargar solo los primeros 5 archivos de ejemplo
for url in urls:
    contenido=descargar_excel(url[0])
     #asignacion de nombre de formato 1
    if contenido != '1' and url[1]=='1':
        print(f"Descargado: {extensionglobal}")
        nombre_archivo = extensionglobal.split('mayoristas_')[-1]   # Obtiene la última parte de la URL
        nombre_archivo = nombre_archivo.split('_')
        final = nombre_archivo[2].split('.')
        nombre_archivo=final[0]+'_'+remplazar2[nombre_archivo[0]]+'_'+nombre_archivo[1]+"."+final[1]
        print(nombre_archivo)
        # Guardar el archivo descargado con el nombre correspondiente
        with open(nombre_archivo, 'wb') as f:
            f.write(contenido)
    
    #asignacion de nombre de formato 2
    elif contenido != '1' and url[1]=='2':
        print(f"Descargado: {extensionglobal}")
        nombre_archivo = extensionglobal.split('anex_')[-1]   # Obtiene la última parte de la URL
        bandera = nombre_archivo.split('_')[0]
        #toma el caso raro donde se ponen 2 meses al principio
        if bandera == 'may':
            nombre_archivo = nombre_archivo.replace("may_feb",'02')
        #caso normal
        else:
            nombre_archivo = nombre_archivo.replace(f'{bandera}',f'{remplazar[bandera]}')
        #nombre segun formato iso
        nombre_archivo = nombre_archivo.split('_')
        final = nombre_archivo[2].split('.')
        nombre_archivo=final[0]+'_'+nombre_archivo[0]+'_'+nombre_archivo[1]+"."+final[1]
        print(nombre_archivo)
        # Guardar el archivo descargado con el nombre correspondiente
        with open(nombre_archivo, 'wb') as f:
            f.write(contenido)
    #asignacion de nombre de formato 3
    elif contenido != '1' and url[1]=='3':
        print(f"Descargado: {extensionglobal}")
        nombre_archivo = extensionglobal.split('SIPSADiario-')[-1]   # Obtiene la última parte de la URL
        #sacamos la informacion y la ponemos en el formato estandar
        fecha = nombre_archivo[:2]
        mes = nombre_archivo[2:5]
        ano = nombre_archivo[5:9]
        extension = nombre_archivo[9:]
        nombre_archivo = ano+'_'+remplazar[mes]+'_'+fecha+extension
        # Guardar el archivo descargado con el nombre correspondiente
        with open(nombre_archivo, 'wb') as f:
            f.write(contenido)
   
    else:
        print(f"Fallo en descargar: {url}")

KeyboardInterrupt: 

In [ ]:
# Especifica la ruta del archivo Excel
archivo_excel = 'febrero_16_2022.xlsx'
archivo_excel1 = 'junio_17_2022.xlsx'

# Lee el archivo Excel en un DataFrame de Pandas
df = pd.read_excel(archivo_excel,header=2)
df1 = pd.read_excel(archivo_excel1,header=2)

# Ahora puedes trabajar con el DataFrame 'df'
# Por ejemplo, imprimir las primeras filas del DataFrame
#df.iloc[:, [0, 21]]
dflimpio1 =df.loc[:, ["Precio $/Kg","Tunja"]]
dflimpio1 =dflimpio1.rename(columns={ 'Tunja': 'junio_26_2018'})
dflimpio2 =df1.loc[:, ["Precio $/Kg","Tunja"]]

dflimpio1 = dflimpio1.drop([0,1,13,33,40,41,42])
dflimpio2 = dflimpio2.drop([0,1,13,33,40,41,42])


dfinal  =pd.merge(dflimpio1, dflimpio2, on='Precio $/Kg', how='left') 
dfinal


KeyError: "['Tunja'] not in index"

In [5]:
import pandas as pd
import glob
import os

# Directorio donde se encuentran los archivos Excel
directorio = ''

# Lista para almacenar los DataFrames limpios
dataframes = []

# Iterar sobre todos los archivos .xlsx y .xls en el directorio
for archivo in glob.glob(os.path.join(directorio, '*.xls*')):
    try:
        # Leer el archivo Excel
        df = pd.read_excel(archivo, header=2)
        
        # Verificar si existe la columna 'Tunja'
        if 'Tunja' in df.columns:
            # Limpiar el DataFrame
            dflimpio = df.loc[:, ["Precio $/Kg", "Tunja"]]
            nombre_archivo = os.path.basename(archivo).split('.')[0]  # Obtener el nombre del archivo sin extensión
            dflimpio = dflimpio.rename(columns={'Tunja': nombre_archivo})
            dflimpio = dflimpio.drop([0, 1, 13, 33, 40, 41, 42])
            
            # Añadir el DataFrame limpio a la lista
            dataframes.append(dflimpio)
    except Exception as e:
        print(f'Error procesando {archivo}: {e}')
        continue

# Combinar todos los DataFrames limpios en uno solo
dfinal = dataframes[0]
for df in dataframes[1:]:
    dfinal = pd.merge(dfinal, df, on='Precio $/Kg', how='left')

# Mostrar el DataFrame final
print(dfinal)

Error procesando 2018_05_08.xlsx: '[42] not found in axis'
                Precio $/Kg 2018_05_10 2018_05_11 2018_05_15 2018_05_17  \
0                   Ahuyama        625        625        625        625   
1     Arveja verde en vaina       3900       3833       3900       4700   
2   Cebolla cabezona blanca        558        550        470        400   
3             Cebolla junca        867        783        833        900   
4           Chocolo mazorca       1330       1600       1200       1450   
5             Fríjol verde*       3000       3050       2800       3050   
6                Habichuela       1740       1483       1600       1683   
7           Lechuga batavia       1517       1458       1600       1600   
8           Pepino cohombro        640        600        640        600   
9                  Pimentón       2786       2679       2857       2500   
10                Remolacha        783        800        820       1000   
11                Zanahoria       1667   

In [6]:
# Re limpieza basado en la visualizacion del dataframe
#dropeamos alimentos con muchos valores faltantes.
dfinal = dfinal.drop([12,15,31,32,33,35])

print(dfinal)

                Precio $/Kg 2018_05_10 2018_05_11 2018_05_15 2018_05_17  \
0                   Ahuyama        625        625        625        625   
1     Arveja verde en vaina       3900       3833       3900       4700   
2   Cebolla cabezona blanca        558        550        470        400   
3             Cebolla junca        867        783        833        900   
4           Chocolo mazorca       1330       1600       1200       1450   
5             Fríjol verde*       3000       3050       2800       3050   
6                Habichuela       1740       1483       1600       1683   
7           Lechuga batavia       1517       1458       1600       1600   
8           Pepino cohombro        640        600        640        600   
9                  Pimentón       2786       2679       2857       2500   
10                Remolacha        783        800        820       1000   
11                Zanahoria       1667       1458       1375       1250   
13               Aguacate

In [7]:
import numpy as np
# Limpiamos los datos(n.d significa no disponible, que para efectos practicos lo tomaremos como nan)
dfinal.replace("n.d.", np.nan, inplace=True)


# teniendo en cuenta que son series de tiempo, tomamos el ultimo dato valido en la serie para remplazar estos datos vacios
dfinal.bfill(inplace=True)


C:\Users\davi8\AppData\Local\Temp\ipykernel_32424\3057375180.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfinal.replace("n.d.", np.nan, inplace=True)


In [8]:
dfinal

,Precio $/Kg,2018_05_10,2018_05_11,2018_05_15,2018_05_17,2018_05_18,2018_05_22,2018_05_24,2018_05_25,2018_05_29,...,2024_07_09,2024_07_11,2024_07_12,2024_07_16,2024_07_18,2024_07_19,2024_07_23,2024_07_25,2024_07_26,2024_07_30
0,Ahuyama,625.0,625,625.0,625.0,594,625.0,594.0,547.0,594.0,...,875.0,917.0,938.0,938.0,958.0,979.0,896.0,844.0,896.0,958.0
1,Arveja verde en vaina,3900.0,3833,3900.0,4700.0,4700,4233.0,4233.0,4180.0,5000.0,...,5700.0,4867.0,4733.0,5800.0,5850.0,5533.0,4867.0,4800.0,5067.0,4450.0
2,Cebolla cabezona blanca,558.0,550,470.0,400.0,430,393.0,430.0,400.0,380.0,...,3733.0,3467.0,2725.0,2867.0,3267.0,3233.0,2633.0,2133.0,1633.0,2025.0
3,Cebolla junca,867.0,783,833.0,900.0,917,911.0,867.0,833.0,950.0,...,1875.0,1958.0,1917.0,1944.0,1944.0,1889.0,1917.0,1958.0,1889.0,1917.0
4,Chocolo mazorca,1330.0,1600,1200.0,1450.0,1358,1300.0,1400.0,1270.0,1200.0,...,5867.0,5933.0,5800.0,5733.0,5667.0,5533.0,5133.0,5000.0,4000.0,4067.0
5,Fríjol verde*,3000.0,3050,2800.0,3050.0,3117,3040.0,2833.0,3100.0,3200.0,...,5867.0,5933.0,5800.0,5733.0,5667.0,5533.0,5133.0,5000.0,4000.0,4067.0
6,Habichuela,1740.0,1483,1600.0,1683.0,1980,2467.0,2100.0,1683.0,2067.0,...,2467.0,2600.0,2733.0,2733.0,2773.0,2667.0,2800.0,2830.0,3000.0,3133.0
7,Lechuga batavia,1517.0,1458,1600.0,1600.0,1550,1567.0,1200.0,1200.0,1250.0,...,1378.0,1467.0,1378.0,1422.0,1511.0,1800.0,1911.0,1956.0,1800.0,1644.0
8,Pepino cohombro,640.0,600,640.0,600.0,600,800.0,800.0,800.0,1000.0,...,1280.0,1173.0,1080.0,1040.0,1107.0,1360.0,1253.0,1200.0,1107.0,1227.0
9,Pimentón,2786.0,2679,2857.0,2500.0,2440,2500.0,2292.0,2649.0,2679.0,...,4405.0,4107.0,4196.0,3929.0,3690.0,3524.0,2976.0,2619.0,2571.0,2643.0


In [15]:
#exportamos
dtra = dfinal.T

KeyError: "['Precio $/Kg'] not found in axis"

In [17]:
dtra.to_csv("data.csv",header=None, index=False)

In [16]:
dtra

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,34
Precio $/Kg,Ahuyama,Arveja verde en vaina,Cebolla cabezona blanca,Cebolla junca,Chocolo mazorca,Fríjol verde*,Habichuela,Lechuga batavia,Pepino cohombro,Pimentón,...,Mango tommy,Manzana royal gala,Maracuyá,Mora de Castilla,Naranja*,Papaya maradol,Piña *,Tomate de árbol,Arracacha*,Plátano hartón verde
2018_05_10,625.0,3900.0,558.0,867.0,1330.0,3000.0,1740.0,1517.0,640.0,2786.0,...,1591.0,4211.0,3050.0,3750.0,783.0,1088.0,800.0,1630.0,1250.0,1250.0
2018_05_11,625,3833,550,783,1600,3050,1483,1458,600,2679,...,1591,4211,3000,3750,800,1194,783,1587,1333,1250
2018_05_15,625.0,3900.0,470.0,833.0,1200.0,2800.0,1600.0,1600.0,640.0,2857.0,...,1591.0,4342.0,2433.0,730.0,730.0,1167.0,1739.0,1739.0,1350.0,1350.0
2018_05_17,625.0,4700.0,400.0,900.0,1450.0,3050.0,1683.0,1600.0,600.0,2500.0,...,1496.0,4474.0,2375.0,660.0,660.0,1278.0,706.0,1891.0,1333.0,1442.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024_07_19,979.0,5533.0,3233.0,1889.0,5533.0,5533.0,2667.0,1800.0,1360.0,3524.0,...,1762.0,7632.0,3533.0,5333.0,1833.0,1275.0,2583.0,3267.0,2194.0,2467.0
2024_07_23,896.0,4867.0,2633.0,1917.0,5133.0,5133.0,2800.0,1911.0,1253.0,2976.0,...,2381.0,7193.0,3433.0,5167.0,1813.0,1233.0,2667.0,3173.0,2111.0,2300.0
2024_07_25,844.0,4800.0,2133.0,1958.0,5000.0,5000.0,2830.0,1956.0,1200.0,2619.0,...,2417.0,7456.0,3300.0,5333.0,1767.0,1325.0,2944.0,3100.0,2111.0,2200.0
2024_07_26,896.0,5067.0,1633.0,1889.0,4000.0,4000.0,3000.0,1800.0,1107.0,2571.0,...,2589.0,7632.0,3067.0,5100.0,1700.0,1300.0,2944.0,3267.0,2250.0,2217.0
